In [103]:
import numpy as np
import pandas as pd

In [104]:
student_data = pd.read_excel("Raw Data/Anon Enrollment Data.xlsx")
enrol_nums_data = pd.read_excel("Raw Data/Course Enrollment Numbers.xlsx", sheet_name="Course Numbers")
enrol_nums_additional = pd.read_excel("Raw Data/Course Enrollment Numbers.xlsx", sheet_name="abbreviated course names")
timetable_data = pd.read_excel("Raw Data/School of Mathematics - Timetable Data.xlsx")
room_data = pd.read_excel("Raw Data/Timetabling KB Rooms_5205.xlsx")

In [105]:
unique_students = student_data["UUN"].unique()
unique_years = student_data["Year Of Programme"].unique()
unique_degrees = student_data["Programme Of Study Sought Title"].unique()
unique_schools = student_data["Programme School Name"].unique()
unique_years_taken = student_data["Normal Year Taken"].unique()
unique_course_codes = student_data['Course Code'].unique()
unique_courses = student_data['Course Name'].unique()

In [106]:
enrol_nums_cols = list(enrol_nums_data.columns)

In [107]:
student_cols = list(student_data.columns)

In [108]:
math_courses_only = student_data[student_data["Course Code"].str.startswith("MATH")]
math_courses = math_courses_only["Course Name"].unique()

enrol_nums_courses = list(enrol_nums_data["Course"])
uncommon_maths_courses = [i for i in math_courses if i not in enrol_nums_courses]

In [109]:
with open('course_abbreviation_key.txt', 'r') as file:
    # Read the entire contents of the file
    abbr_key = file.read()
file.close()

abbr_key = abbr_key.split("\n")
abbr_key = [i.strip() for i in abbr_key if i.strip()]

In [110]:
enrol_nums_copy = enrol_nums_data.copy()

In [111]:
abbr_map = {}
for course in abbr_key:
    abbreviation, description, code = course.split(' = ')
    abbr_map[code] = abbreviation

enrol_nums_copy["Abbreviated Course Name"] = enrol_nums_copy["Code"].map(abbr_map)

In [112]:
len(enrol_nums_copy["Abbreviated Course Name"]) == len(enrol_nums_additional["Abbreviated Course Name"])

True

In [113]:
enrol_nums_copy["Abbreviated Course Name"].all() == enrol_nums_additional["Abbreviated Course Name"].all()

True

# Changing Room Data

In [249]:
room_copy = room_data.copy()
new_room_cols = ["campus", "building", "room_name", "capacity",
                 "ownership", "gt_or_ts", "school_priority", "room_layout","furniture_config",
                 "control_system", "desktop", "induction_system", "lec_recording", "microphone", "pres_facilities", 
                 "sound_system", "wall_mounted_writing"]

room_copy.columns = new_room_cols

In [250]:
unique_campuses = room_copy["campus"].unique()
unique_buildings = room_copy["building"].unique()
unique_rooms = room_copy["room_name"].unique()
unique_ownership = room_copy["ownership"].unique()

In [251]:
if len(unique_campuses)==1:
    if unique_campuses.all() == "King's Buildings Campus":
        room_copy["campus"] = room_copy["campus"].replace("King's Buildings Campus", "KB")
else:
    print("There is more than one campus in the dataset.")


building_replacements = {"Alrick":"ALR",
                         "Ashworth":"ASH",
                         "Daniel Rutherford Building":"DRB",
                         "Eng Sanderson Building":"ESB",
                         "Grant Institute":"GRA",
                         "Hudson Beare Building":"HBB",
                         "Joseph Black Building":"JBB",
                         "JCMB":"JCMB",
                         "Murchison House":"MH",
                         "Nucleus":"NUC",
                         "Swann Building":"SB"}

room_copy["building"] = room_copy["building"].replace(building_replacements)


allocation_replacements = {"1. Centrally Allocated Space":"centrally_allocated", "1. Locally Allocated Space": "locally_allocated"}
room_copy["ownership"] = room_copy["ownership"].replace(allocation_replacements)


gt_ts_replacements = {"2. General Teaching":"GT", "2. Teaching Studio":"TS"}
room_copy["gt_or_ts"] = room_copy["gt_or_ts"].replace(gt_ts_replacements)

In [252]:
room_copy["school_priority"] = room_copy["school_priority"].str.split(" - ").str[1]
room_copy["room_layout"] = room_copy["room_layout"].str.split(" - ").str[1]

In [253]:
room_copy["school_priority"] = room_copy["school_priority"].fillna("no priority")

priority_replacements = {"Engineering":"eng",
                         "Biological":"bio",
                         "Geosciences":"geo",
                         "Chemistry":"chem",
                         "Mathematics":"maths",
                         "Physics and Astronomy":"phys",
                         "Mathematics/ Physics and Astronomy":"maths_phys"}
room_copy["school_priority"] = room_copy["school_priority"].replace(priority_replacements)


# room_copy["room_layout"] = room_copy["room_layout"].fillna("")
layout_replacements = {"Classroom Style":"classroom",
                       "Theatre Style":"theatre",
                       "Boardroom Style":"boardroom"}
room_copy["room_layout"] = room_copy["room_layout"].replace(layout_replacements)

In [254]:
room_copy["furniture_config"] = room_copy["furniture_config"].str.split("\. ").str[1]

In [255]:
furniture_replacements = {"Tables and Chairs":"tables_chairs",
                          "Flexible seating (not tables)":"flex_seating",
                          "Retractable Seating":"retract_seating"}
room_copy["furniture_config"] = room_copy["furniture_config"].replace(furniture_replacements)

In [256]:
list(room_copy["furniture_config"].unique())

['tables_chairs', 'flex_seating', nan, 'retract_seating']

In [257]:
true_false_cols = ["control_system", "desktop", "induction_system", "lec_recording", "microphone", "pres_facilities", "sound_system", "wall_mounted_writing"]

for col in true_false_cols:
    room_copy[col] = room_copy[col].apply(lambda x: True if pd.notna(x) and x.strip() else False)

# room_copy["control_system"] = room_copy["control_system"].apply(lambda x: True if pd.notna(x) and x.strip() else False)
# room_copy["desktop"] = room_copy["desktop"].apply(lambda x: True if pd.notna(x) and x.strip() else False)
# room_copy["induction_system"] = room_copy["induction_system"].apply(lambda x: True if pd.notna(x) and x.strip() else False)
# room_copy["lec_recording"] = room_copy["lec_recording"].apply(lambda x: True if pd.notna(x) and x.strip() else False)
# room_copy["microphone"] = room_copy["microphone"].apply(lambda x: True if pd.notna(x) and x.strip() else False)
# room_copy["pres_facilities"] = room_copy["pres_facilities"].apply(lambda x: True if pd.notna(x) and x.strip() else False)
# room_copy["sound_system"] = room_copy["sound_system"].apply(lambda x: True if pd.notna(x) and x.strip() else False)
# room_copy["wall_mounted_writing"] = room_copy["wall_mounted_writing"].apply(lambda x: True if pd.notna(x) and x.strip() else False)

In [258]:
dummy_enc_cols = []


room_furniture_encoded = pd.get_dummies(room_copy["furniture_config"], prefix="furniture")
room_layout_encoded = pd.get_dummies(room_copy["room_layout"], prefix="layout")
gt_ts_encoded = pd.get_dummies(room_copy["room_layout"], prefix="layout")

room_copy = pd.concat([room_copy, room_encoded], axis=1)
room_copy = room_copy.drop("furniture_config", axis=1)



In [260]:
# room_copy.drop("furniture_config", axis=1)
list(room_copy.columns)

room_copy

,campus,building,room_name,capacity,ownership,gt_or_ts,school_priority,room_layout,control_system,desktop,induction_system,lec_recording,microphone,pres_facilities,sound_system,wall_mounted_writing,furniture_flex_seating,furniture_retract_seating,furniture_tables_chairs
0,KB,ALR,ALR_Classroom 10,86,centrally_allocated,GT,eng,classroom,True,True,False,True,True,True,True,True,False,False,True
1,KB,ASH,ASH_Honours Room 1,35,centrally_allocated,GT,bio,classroom,True,True,False,True,True,True,True,True,True,False,False
2,KB,ASH,ASH_Honours Room 2,33,centrally_allocated,GT,bio,classroom,True,True,False,True,True,True,True,True,True,False,False
3,KB,ASH,ASH_Lecture Theatre 1,313,centrally_allocated,GT,bio,theatre,False,True,True,True,False,True,True,True,False,False,False
4,KB,ASH,ASH_Lecture Theatre 3,88,centrally_allocated,GT,bio,theatre,False,True,True,True,False,True,True,True,False,False,False
5,KB,DRB,DRB_G.27 Lecture Theatre 1,164,centrally_allocated,GT,bio,theatre,False,True,True,True,False,True,True,True,False,False,False
6,KB,ESB,ESB_Lecture Theatre No. 1,200,centrally_allocated,GT,eng,theatre,False,True,False,True,False,True,True,True,False,False,False
7,KB,GRA,GRA_Lecture Theatre 201,135,centrally_allocated,GT,geo,theatre,False,True,True,True,True,True,True,True,False,False,False
8,KB,GRA,GRA_Room 304B,40,centrally_allocated,GT,geo,classroom,False,True,False,True,False,True,True,True,True,False,False
9,KB,HBB,HBB_Classroom 4,54,centrally_allocated,GT,eng,classroom,False,True,False,True,True,True,True,True,False,False,True
